In [1]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import seaborn as sns
import sys
import warnings
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from collections import Counter

nltk.download('stopwords')

#Location of your training set
df = pd.read_csv(r"C:\Users\Leo\Desktop\Thesis\Flask Thesis\uploads\dataset for labeling - emails.csv")
#Location of your testing set
testdf = pd.read_csv(r"C:\Users\Leo\Desktop\Thesis\Flask Thesis\uploads\file.csv")

df.fillna(0, inplace=True)
index = df[ df['SUM'] == 0 ].index
df.drop(index, inplace = True)

testdf['from'] = testdf['from'].replace({'=?':''},regex=True)

testdf.body = testdf.body.replace({'CONFIDENTIALITY NOTICE:.*blueboard':''},regex=True)

df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,subject,from,date,body,Category,Label,Label 2 (if applicable),USG0,USG1,USG2,USG5,SUM
0,"[Blueboard] Officer-in-Charge, Loyola Schools ...","""Office of the Assoc. Dean for Student & Admin...","Fri, 19 Oct 2018 12:09:24 -0700","To the Loyola Schools Community,\n\n\n\n\n\nPl...",Leave/OIC,USG0,0,1.0,0.0,0.0,0.0,1
1,[Blueboard] School of Humanities Research Load...,"""Office of the Dean [LS]"" <dean.soh@ateneo.edu>","Fri, 19 Oct 2018 11:33:05 +0800",The SOH Research Committee and the SOH Office ...,Talk/Lecture/Lecture Series/Discussion/Workshop,USG1,0,0.0,1.0,0.0,0.0,1
2,[Blueboard] ASOG Baccalaureate Mass and Commen...,"""ASoG Academic Program [ASOG]"" <academic.asog@...","Fri, 12 Oct 2018 12:44:39 +0800",[image: 20181012 Bacc Mass and Commencement An...,Other,USG0,0,1.0,0.0,0.0,0.0,1
3,[Blueboard] Congratulations to all our finalis...,"""Karla C. Mercado"" <kmercado@ateneo.edu>","Wed, 3 Oct 2018 09:05:10 +0800",[image: CongratsFinalistsNBA2018_BlueBrd-02 Fi...,Other,USG0,0,1.0,0.0,0.0,0.0,1
4,[Blueboard] Explore Ateneo: A Walk on the Wild...,"""Employee Engagement [HRMOD]"" <employee.engage...","Wed, 26 Sep 2018 14:49:54 +0800",Did you know that only around 30% of the campu...,Program/Course,USG1,0,0.0,1.0,0.0,0.0,1


In [2]:
df.drop(labels = ['Label','Label 2 (if applicable)', 'date'], axis = 1, inplace=True)

In [3]:
#Cleaning

if not sys.warnoptions:
    warnings.simplefilter("ignore")
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent


In [4]:
df['content'] = df['subject'].astype(str) + " " + df['body'].astype(str)
df['content'] = df['content'].str.lower()
df['content'] = df['content'].apply(cleanHtml)
df['content'] = df['content'].apply(cleanPunc)
df['content'] = df['content'].apply(keepAlpha)

testdf['content'] = testdf['subject'].astype(str) + " " + df['body'].astype(str)
testdf['content'] = testdf['content'].str.lower()
testdf['content'] = testdf['content'].apply(cleanHtml)
testdf['content'] = testdf['content'].apply(cleanPunc)
testdf['content'] = testdf['content'].apply(keepAlpha)

stop_words = set(stopwords.words('english'))
stop_words.update(["ls","www","university", "ateneo", "thank", "mvp", "pm", "email", "loyola", "write","http","list","univers","grade", "dismiss", "traffic", "bulletin", "school", "high", "office", "inform", "mr", "mrs", "ms", "edu", "dr", "blueboard", "close", "prayer", "wish", "calibri","font","style","margin","span","color","ampc"])
stop_words.update(["px", "width", "media", "height", "max", "https", "de", "manila","size","background","line", "helvetica", "import"])
stop_words.update(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'])
stop_words.update(["id", "df", "ff", "top", "bottom", "border", "center", "mcntextcont", "weight" ,"align", "text", "utf", "also"])
stop_words.update(["monday", "tuesday", "wednesday", "thursday", "friday", "january", "february", "march", "april", "may", "june", "july","august","september","october","november","december"])
stop_words.update(["please", "pleas"])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)
df['content'] = df['content'].apply(removeStopWords)
testdf['content'] = testdf['content'].apply(removeStopWords)

stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence
df['content'] = df['content'].apply(stemming)
testdf['content'] = testdf['content'].apply(stemming)

#from sklearn.model_selection import train_test_split
#train, test = train_test_split(df, random_state=42, test_size=0.30, shuffle=True)

#train_text = train['content']
#test_text = test['content']
train_text = df['content']
test_text = testdf['content']

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2', stop_words=stop_words)
vectorizer.fit(train_text)
vectorizer.fit(test_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words={'during', 'university', 'any', 'most', 'd', 'against', 'myself', 'll', 'it', 'from', "shan't", 'mr', 'utf', "aren't", "doesn't", "should've", 'february', 'de', 'top', "haven't", 'https', 'his', 'no', 'n', 'of', 'l', 'hers', 'further', 'then', 'under', 'k', 'p', 'email', "you're", 'should...l', 'pm', 'manila', 'about', 'my', 'shan', 'yourself', "it's", 'them', 'mightn', 'ls', 'the', 'few'},
        strip_accents='unicode', sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [5]:
categories = list(df.columns.values)
categories = categories[4:8]
#train, test = train_test_split(df, random_state=42, test_size=0.33, shuffle=True)

X_train = df.content
X_test = testdf.content

In [6]:
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])

In [7]:
SVC_pipeline.fit(X_train, df[categories[0]])
usg0prediction = SVC_pipeline.predict(X_test)

SVC_pipeline.fit(X_train, df[categories[1]])
usg1prediction = SVC_pipeline.predict(X_test)

SVC_pipeline.fit(X_train, df[categories[2]])
usg2prediction = SVC_pipeline.predict(X_test)

SVC_pipeline.fit(X_train, df[categories[3]])
usg5prediction = SVC_pipeline.predict(X_test)

In [8]:
X_test.head(100)

0     offic charg associ dean student format school ...
1     art educ everi child soh research committe soh...
2     extramur class imag bacc mass commenc announce...
3     offic charg scienc engin imag congratsfinalist...
4     philippin govern anti drug campaign examin leg...
5     develop field invit everyon join us pray repos...
6     st sociolog anthropolog student research confe...
7     ohrmod oper japanes studi program invit everyo...
8     archiv schedul ricardo leong chines studi clos...
9     talk tradit cultur landscap philippin america ...
10    remind council meet abstract famili farm toget...
11    lead ladi song music theater concert tue escal...
12    menu week inquiri confirm attend contact insti...
13    central purchas close everyon invit visit somb...
14    happen today critic think cathol religi educ s...
15    henri lee irwin sj chair lectur stage adapt ro...
16    chang date venu peopl choic sa eleksyon gabay ...
17    secur alert data protect udpo close meet t

In [9]:
x_test = X_test.to_frame()


#Output Email and its USGs
for number in range(0, x_test.size):
    print("\nEmail: ", x_test.content.iloc[number])
    print("Labels: ")
    if(usg0prediction.item(number) == 1):
        print("USG0")
    if(usg1prediction.item(number) == 1):
        print("USG1")
    if(usg2prediction.item(number) == 1):
        print("USG2")
    if(usg5prediction.item(number) == 1):
        print("USG5")
    if(usg0prediction.item(number) == 0 and usg1prediction.item(number) == 0 and usg2prediction.item(number) == 0 and usg5prediction.item(number) == 0 ):
        print("USG0")
    


Email:  offic charg associ dean student format school communiti advis maria virginia acevedo director school bookstor person leav absenc michael jacinto mallillin director student servic offic charg ordinari busi brought attent lourd mangona administr assist school bookstor inform guidanc everyon much associ dean student administr servic school univers
Labels: 
USG0

Email:  art educ everi child soh research committe soh dean invit first research load fora pmat ngf confer roomground floor dela costa hall faculti present research ruben mendoza contribut jose mario francisco sj theolog philippin jocelyn martin tyrant typhoon trauma spectral magic realism nick joaquin cave shadow maricel ibita plead case mountain ecolog read legal metaphor micah everyon invit research fora rsvp goo gl form gz qcgw xkrfovqx call loc dean human univers
Labels: 
USG1

Email:  extramur class imag bacc mass commenc announcemen admu jpg
Labels: 
USG0

Email:  offic charg scienc engin imag congratsfinalistsnba 

In [30]:
export = pd.DataFrame(columns=['subject', 'body', 'USG0', 'USG1', 'USG2', 'USG5'])

for number in range(0, x_test.size):
    subject = testdf.subject.iloc[number]
    body = testdf.body.iloc[number]
    usg0 = False
    usg1 = False
    usg2 = False
    usg5 = False
    if(usg0prediction.item(number) == 1):
        usg0 = True
    if(usg1prediction.item(number) == 1):
        usg1 = True
    if(usg2prediction.item(number) == 1):
        usg2 = True
    if(usg5prediction.item(number) == 1):
        usg5 = True
    if(usg0prediction.item(number) == 0 and usg1prediction.item(number) == 0 and usg2prediction.item(number) == 0 and usg5prediction.item(number) == 0 ):
        usg0 = True
    export = export.append({'subject': subject, 'body': body, 'USG0': usg0, 'USG1': usg1, 'USG2': usg2, 'USG5': usg5}, ignore_index=True)
        
export.head()
export.to_csv(r'')

,subject,body,USG0,USG1,USG2,USG5
0,"[Blueboard] Officer-in-Charge,\r\n Office of t...",[image: Inline image 1]\r\n\r\n22 April 2019\r...,True,False,False,False
1,[Blueboard] ART for EDUCATION: Every Child Can,"Mano Amiga Academy Philippines, World Dance Al...",False,True,False,False
2,[Blueboard] Extramural Classes,*BASIC KOREAN LANGUAGE AND CULTURE I*\r\n\r\n2...,True,False,False,False
3,"[Blueboard] Officer-in-Charge,\r\n\tSchool of ...",[image: Inline image 1]\r\n\r\n22 April 2019\r...,True,False,False,False
4,[Blueboard] The Philippine Government's Anti-D...,[image: 2nd FB layout_v4-min (1).png]\r\n\r\nO...,False,True,False,False


In [10]:
#Corpus
#Show top words associated w each USG

#USG1
usg1email = pd.DataFrame(columns=['content'])

for number in range(0, x_test.size):
    if(usg1prediction.item(number) == 1):
        usg1email = usg1email.append(x_test.iloc[number])

count = Counter(" ".join(usg1email["content"]).split()).most_common(100)
usg1corpus = pd.DataFrame(count, columns=["word", "count"])

index = usg1corpus[ usg1corpus['word'].str.contains('import', case = False) == True ].index
usg1corpus.drop(index, inplace = True)
index = usg1corpus[ usg1corpus['word'].str.contains('mcntextcont', case = False) == True ].index
usg1corpus.drop(index, inplace = True)

usg1corpus.head(20)

,word,count
1,art,554
2,philippin,548
3,korean,517
4,studi,442
6,screen,376
7,pad,352
8,depart,340
9,program,332
10,us,318
11,cours,310


In [11]:
#USG2
usg2email = pd.DataFrame(columns=['content'])

for number in range(0, x_test.size):
    if(usg2prediction.item(number) == 1):
        usg2email = usg2email.append(x_test.iloc[number])

count = Counter(" ".join(usg2email["content"]).split()).most_common(100)
usg2corpus = pd.DataFrame(count, columns=["word", "count"])

#index = usg1corpus[ usg1corpus['word'].str.contains('import', case = False) == True ].index
#usg1corpus.drop(index, inplace = True)
#index = usg1corpus[ usg1corpus['word'].str.contains('mcntextcont', case = False) == True ].index
#usg1corpus.drop(index, inplace = True)

usg2corpus.head(20)

,word,count
0,studi,268
1,philippin,189
2,program,171
3,korean,138
4,social,133
5,research,132
6,student,128
7,cultur,118
8,citi,112
9,school,102


In [12]:
#USG5
usg5email = pd.DataFrame(columns=['content'])

for number in range(0, x_test.size):
    if(usg5prediction.item(number) == 1):
        usg5email = usg5email.append(x_test.iloc[number])

count = Counter(" ".join(usg5email["content"]).split()).most_common(100)
usg5corpus = pd.DataFrame(count, columns=["word", "count"])

#index = usg1corpus[ usg1corpus['word'].str.contains('import', case = False) == True ].index
#usg1corpus.drop(index, inplace = True)
#index = usg1corpus[ usg1corpus['word'].str.contains('mcntextcont', case = False) == True ].index
#usg1corpus.drop(index, inplace = True)

usg5corpus.head(20)

,word,count
0,donat,213
1,philippin,190
2,name,131
3,address,121
4,social,117
5,payment,101
6,citi,100
7,account,100
8,us,96
9,disast,91


,content,USG0,USG1,USG2,USG5
